# 8. Resolve data issues

In [1]:
sample_file = '../preprocessed_data/sample_2019-08-17.feather'
issues_to_address = '../preprocessed_data/issues_to_address_2019-08-17.pickle'
pipelines_2010_raw_file = '../data/pipelines_2010_2019-08-16.feather'

## Setup

In [2]:
import pandas as pd
import numpy as np
from datetime import date
from functools import partial

today = date.today().isoformat()

In [3]:
import wrds

db = wrds.Connection(wrds_username='juujian')

Loading library list...
Done


## Load data

In [4]:
import pickle

with open(f'../preprocessed_data/issues_to_address_2019-08-14.pickle', 'rb') as file:
    issues = pickle.load(file)
    
issues = (issue for issue in issues)

In [5]:
sample = pd.read_feather(sample_file)
sample.sample(5)

,OPERATOR_ID,YEAR,NAME,COMMODITY,MILES,PARENT,INCIDENTS,SIGNIFICANT_INCIDENTS
126,4906,2014,EXXONMOBIL PIPELINE CO,non-hvl,536.84,Exxon Mobil,3.0,1.0
739,32334,2011,TC OIL PIPELINE OPERATIONS INC,crude,524.00,TransCanada USA,6.0,4.0
301,22610,2017,"MAGELLAN PIPELINE COMPANY, LP",non-hvl,3925.12,Magellan Midstream Partners,9.0,1.0
114,4805,2010,EXPLORER PIPELINE CO,non-hvl,1221.02,Explorer Pipeline,7.0,5.0
136,4906,2018,EXXONMOBIL PIPELINE CO,crude,366.51,Exxon Mobil,3.0,1.0


In [6]:
pipelines_2010_raw = pd.read_feather(pipelines_2010_raw_file)

## 8.1 Functions for analysis

In [7]:
from functools import partial

def find_info(OPERATOR_ID, info_col: str, title: str, df = pipelines_2010_raw, id_col = 'OPERATOR_ID', 
              year_col = 'REPORT_YEAR', fuzzy=False):
    from fuzzywuzzy import fuzz
    
    values = np.unique(df[df[id_col] == int(OPERATOR_ID)][info_col]).tolist()
    result = []
    for value in values:
        start_year = df[df[info_col] == value][year_col].min()
        end_year = df[df[info_col] == value][year_col].max()
        result = result + [{title: value, 'start_year': start_year, 'end_year': end_year}]
        
    if fuzzy and len(result) == 2 and fuzz.ratio(result[0][title].lower(), result[1][title].lower()) >= 95:
            result = [result[0]]
            
    return(result)

find_address = partial(find_info, info_col='PARTA4STREET', title='address')
find_names = partial(find_info, info_col='PARTA2NAMEOFCOMP', title='name', fuzzy=True)
find_names('4906')

[{'name': 'EXXONMOBIL PIPELINE CO', 'start_year': 2010, 'end_year': 2018}]

In [8]:
def find_total_miles_year(OPERATOR_ID, year, df = pipelines_2010_raw, year_col = 'REPORT_YEAR', 
                          id_col = 'OPERATOR_ID', miles_col='PARTBHCATOTAL'):
    observations = df.loc[(df[year_col] == year) & (df[id_col] == int(OPERATOR_ID))][miles_col]
    return observations.sum()

find_total_miles_year('4906', 2015)

2113.75

In [9]:
def find_total_miles(OPERATOR_ID, years, df = pipelines_2010_raw, year_col = 'REPORT_YEAR', miles_col='PARTBHCATOTAL'):
    result = []
    for year in years:
        result.append([year, find_total_miles_year(OPERATOR_ID, year)])
    return(result)

find_total_miles('4906', range(2013, 2017))

[[2013, 2944.4], [2014, 2343.06], [2015, 2113.75], [2016, 2590.17]]

## 8.2 Iterate over issues and resolve them one at a time

We use the dictionary "company groups" to capture any companies we have to combine to form one observation.

### Store results in

In [10]:
company_groups = pd.DataFrame()

def add_company_group(name: str, members: list, df = company_groups):
    new_group = pd.DataFrame({'members': members})
    new_group['name'] = name
    df = df.append(new_group, ignore_index=True)
    return df

In [11]:
m_as = pd.DataFrame()

def add_m_a(name: str, members: list, df, start_year: str = None, end_year: str = None):
    new_group = pd.DataFrame({'members': members})
    new_group['name'] = name
    
    if start_year:
        new_group['start_year'] = start_year
    if end_year:
        new_group['end_year'] = end_year
    
    df = df.append(new_group, sort=False, ignore_index=True)
    return df

In [12]:
spin_offs = []

### 8.2.1 Enterprise Products Operating

In [13]:
print(next(issues))


ENTERPRISE PRODUCTS OPERATING LLC (OPERATOR_ID 31618) has the same parent company as:

	ENTERPRISE CRUDE PIPELINE LLC (OPERATOR_ID 30829)

	DIXIE PIPELINE COMPANY LLC (OPERATOR_ID 3445)



### Gather infor_asation

In [14]:
find_names('31618')

[{'name': 'ENTERPRISE PRODUCTS OPERATING LLC',
  'start_year': 2010,
  'end_year': 2018}]

In [15]:
find_names('30829')

[{'name': 'ENTERPRISE CRUDE PIPELINE LLC',
  'start_year': 2010,
  'end_year': 2018}]

In [16]:
find_names('3445')

[{'name': 'DIXIE PIPELINE', 'start_year': 2010, 'end_year': 2010},
 {'name': 'DIXIE PIPELINE COMPANY LLC', 'start_year': 2011, 'end_year': 2018}]

In [17]:
db.raw_sql("SELECT * FROM compa.names WHERE lower(conm) LIKE 'enterprise%%' LIMIT 10")

,gvkey,conm,tic,cusip,cik,sic,naics,gsubind,gind,year1,year2
0,004399,ENTERPRISE DEVELOPMENT GROUP,EDG.2,293705109,0000020236,1531,None,None,None,1970.0,1981.0
1,004400,ENTERPRISE TECHNOLOGIES -OLD,ENTRQ,293799102,None,5171,422710,10102030,101020,1981.0,1983.0
2,013370,ENTERPRISE INFORMATICS INC,EINF,29372R208,0000813747,7373,541512,45103010,451030,1986.0,2007.0
3,020151,ENTERPRISE SOFTWARE INC,ENSW,29379K107,0000833847,7372,511210,45103010,451030,1988.0,1998.0
4,030803,ENTERPRISE FED BANCORP INC,EFBI.1,293711107,0000922036,6035,522120,40101010,401010,1994.0,1998.0
5,061438,ENTERPRISE SYSTEMS INC,ESIX,293797106,0001000188,7373,541512,45103020,451030,1990.0,1996.0
6,100571,ENTERPRISE OIL PLC,ETPOY,293779302,0000887985,1311,211111,10102020,101020,1989.0,2001.0
7,112876,ENTERPRISE PRODS PRTNRS -LP,EPD,293792107,0001061219,1311,486210,10102040,101020,1995.0,2018.0
8,117141,ENTERPRISE FINL SERVICES CP,EFSC,293712105,0001025835,6020,522110,40101015,401010,1998.0,2018.0
9,118525,ENTERPRISE BANCORP INC/MA,EBTC,293668109,0001018399,6020,522110,40101015,401010,2002.0,2018.0


In [18]:
db.raw_sql("SELECT * FROM compa.names WHERE lower(conm) LIKE 'dixie%%' LIMIT 10")

,gvkey,conm,tic,cusip,cik,sic,naics,gsubind,gind,year1,year2
0,004001,DIXIE GROUP INC,DXYN,255519100,0000029332,2273,314110,25201020,252010,1974.0,2018.0


### Decision

ENTERPRISE CRUDE PIPELINE LLC (OPERATOR_ID 30829) is a subsidiary of Enterprise Products Operating (https://www.sec.gov/Archives/edgar/data/1061219/000106121918000008/exhibit21_1.htm).

A quick google search reveals that Dixie Group is a "Carpet and rug mills company", so we can disregard this hit and just merge Dixie Pipeline with its parent company.

In [19]:
company_groups = add_company_group('Enterprise Products (Group)', ['31618', '30829', '3445'], company_groups)
company_groups

,members,name
0,31618,Enterprise Products (Group)
1,30829,Enterprise Products (Group)
2,3445,Enterprise Products (Group)


### 8.2.2 Oneok NGL

In [20]:
print(next(issues))


ONEOK NGL PIPELINE, LLC (OPERATOR_ID 32109) has changed its name:

	Was named ONEOK NGL PIPELINE LP from 2010 to 2011.

	Was named ONEOK NGL PIPELINE, LLC from 2012 to 2018.



No action necessary.

### 8.2.3 Phillips 66

In [21]:
print(next(issues))


PHILLIPS 66 PIPELINE LLC (OPERATOR_ID 31684) has changed its name:

	Was named CONOCOPHILLIPS from 2010 to 2010.

	Was named PHILLIPS 66 PIPELINE LLC from 2011 to 2018.



Downstream business was spun off in 2011/2012.

In [22]:
db.raw_sql("SELECT * FROM compa.names WHERE lower(conm) LIKE '%%phillips 66%%' LIMIT 10")

,gvkey,conm,tic,cusip,cik,sic,naics,gsubind,gind,year1,year2
0,017933,PHILLIPS 66 PARTNERS LP,PSXP,718549207,0001572910,4610,486110,10102040,101020,2011.0,2018.0
1,170841,PHILLIPS 66,PSX,718546104,0001534701,2911,324110,10102030,101020,2009.0,2018.0


The first row is a good hit in terms of the naics.

In [23]:
db.raw_sql("SELECT * FROM compa.names WHERE lower(conm) LIKE '%%conocophillips%%' LIMIT 10")

,gvkey,conm,tic,cusip,cik,sic,naics,gsubind,gind,year1,year2
0,008549,CONOCOPHILLIPS,COP,20825C104,0001163165,1311,2111,10102020,101020,1950.0,2018.0
1,150400,CONOCOPHILLIPS-PRO FORMA1,COP.P,20899Y933,None,2911,324110,10102010,101020,2000.0,2001.0
2,166674,CONOCOPHILLIPS-PRO FORMA,COP.R,20899W937,None,2911,324110,10102010,101020,2004.0,2004.0


In [24]:
db.raw_sql("""SELECT gvkey, datadate, fyear, revt AS revenue, act AS assets FROM compa.funda WHERE gvkey='008549'
           AND fyear BETWEEN '2009' AND '2013' AND act is not null""")

,gvkey,datadate,fyear,revenue,assets
0,008549,2009-12-31,2009.0,136016.0,21167.0
1,008549,2010-12-31,2010.0,175752.0,34660.0
2,008549,2011-12-31,2011.0,230859.0,30218.0
3,008549,2012-12-31,2012.0,57967.0,23989.0
4,008549,2013-12-31,2013.0,54413.0,19023.0


In [25]:
db.raw_sql("""SELECT gvkey, datadate, fyear, revt AS revenue, act AS assets FROM compa.funda WHERE gvkey='017933'
           AND fyear BETWEEN '2009' AND '2013' And act is not null""")

,gvkey,datadate,fyear,revenue,assets
0,017933,2011-12-31,2011.0,76.0,0.4
1,017933,2012-12-31,2012.0,80.1,0.5
2,017933,2013-12-31,2013.0,106.6,439.4


In [26]:
db.raw_sql("""SELECT gvkey, datadate, fyear, revt AS revenue, act AS assets FROM compa.funda WHERE gvkey='170841'
           AND fyear BETWEEN '2009' AND '2013' AND act is not null""")

,gvkey,datadate,fyear,revenue,assets
0,170841,2010-12-31,2010.0,132872.0,14704.0
1,170841,2011-12-31,2011.0,182133.0,13948.0
2,170841,2012-12-31,2012.0,166089.0,17962.0
3,170841,2013-12-31,2013.0,157730.0,19237.0


### Decision

It is hard to tell what is going on. Taking the address could clear the confusion, but we would probably obtain the cleanest results if we cut of the observations for the organization in 2011 (after the subsidiary was spun off and renamed according to the FERC documents, since we are not interested in the effects from spin-offs. If more spin-offs occur, we might choose to control for it through a categorical variable.

In [27]:
spin_offs.append({'name': '31684', 'year': '2011'})

### 8.2.4 Buckeye

In [28]:
print(next(issues))


BUCKEYE PARTNERS, LP (OPERATOR_ID 1845) has the same parent company as:

	BUCKEYE DEVELOPMENT & LOGISTICS, LLC (OPERATOR_ID 31371)



In [29]:
db.raw_sql("SELECT * FROM compa.names WHERE lower(conm) LIKE '%%buckeye%%' LIMIT 10")

,gvkey,conm,tic,cusip,cik,sic,naics,gsubind,gind,year1,year2
0,002451,BUCKEYE INTL INC,BEYE.,118169101,0000015070,3320,None,None,None,1965.0,1979.0
1,013526,BUCKEYE PARTNERS LP,BPL,118230101,0000805022,4610,486910,10102040,101020,1986.0,2018.0
2,024417,BUCKEYE COMMUNICATIONS INC,3BKCIE,118154202,0000822822,5130,4223,25501010,255010,1989.0,1994.0
3,061596,BUCKEYE TECHNOLOGIES INC,BKI.1,118255108,0000899597,2611,322110,15105020,151050,1993.0,2013.0
4,174316,BUCKEYE GP HOLDINGS LP,BGH.2,118167105,0001359055,5172,424720,10102040,101020,2001.0,2009.0


### Decision

We consolidate the two organizations into one.

In [30]:
company_groups = add_company_group('Buckeye (Group)', ['1845', '31371'], company_groups)
company_groups.tail()

,members,name
0,31618,Enterprise Products (Group)
1,30829,Enterprise Products (Group)
2,3445,Enterprise Products (Group)
3,1845,Buckeye (Group)
4,31371,Buckeye (Group)


### 8.2.5 Marathon

In [31]:
print(next(issues))


MARATHON PIPE LINE LLC (OPERATOR_ID 32147) has the same parent company as:

	WOLVERINE PIPELINE CO (OPERATOR_ID 22830)

	TESORO LOGISTICS OPERATIONS, LLC (OPERATOR_ID 38933)

	TESORO LOGISTICS NORTHWEST PIPELINE LLC (OPERATOR_ID 39029)



Marathon acquired Andeavor in 2018.

### Names

In [32]:
find_names('32147')

[{'name': 'MARATHON PIPE LINE LLC', 'start_year': 2010, 'end_year': 2018}]

In [33]:
find_names('22830')

[{'name': 'WOLVERINE PIPELINE CO', 'start_year': 2010, 'end_year': 2018}]

In [34]:
find_names('38933')

[{'name': 'TESORO LOGISTICS OPERATIONS LLC - SOUTHERN CALIFORNIA',
  'start_year': 2012,
  'end_year': 2017},
 {'name': 'TESORO LOGISTICS OPERATIONS, LLC',
  'start_year': 2018,
  'end_year': 2018}]

In [35]:
find_names('39029')

[{'name': 'TESORO LOGISTICS NORTHWEST PIPELINE LLC',
  'start_year': 2013,
  'end_year': 2017}]

### Track miles to find merger date

In [36]:
find_total_miles('32147', range(2015, 2019))

[[2015, 3653.3999999999996],
 [2016, 3596.7999999999997],
 [2017, 3948.0],
 [2018, 4037.2]]

In [37]:
find_total_miles('22830', range(2015, 2019))

[[2015, 513.89], [2016, 513.89], [2017, 396.25], [2018, 513.89]]

In [38]:
find_total_miles('38933', range(2015, 2019))

[[2015, 15.3],
 [2016, 15.5],
 [2017, 14.969999999999999],
 [2018, 930.6999999999999]]

In [39]:
find_total_miles('39029', range(2015, 2019))

[[2015, 466.9], [2016, 484.2], [2017, 484.2], [2018, 0.0]]

It seems the are transferred to one of Tosero's business units, but not reported by the (new) parent.

### Identify gvkeys

In [40]:
db.raw_sql("SELECT * FROM compa.names WHERE lower(conm) LIKE '%%marathon%%' AND year2 > 2000 LIMIT 10")

,gvkey,conm,tic,cusip,cik,sic,naics,gsubind,gind,year1,year2
0,004384,MARATHON PATENT GROUP INC,MARA,56585W401,0001507605,7374,518210,45102020,451020,2011.0,2018.0
1,007017,MARATHON OIL CORP,MRO,565849106,0000101778,1311,2111,10102020,101020,1950.0,2018.0
2,177047,MARATHON PGM CORP,MAR.Z,565851102,None,1000,212,15104020,151040,2004.0,2009.0
3,186042,MARATHON GOLD CORP,MGDPF,56580Q102,0001507472,1000,212221,15104030,151040,2010.0,2018.0
4,186989,MARATHON PETROLEUM CORP,MPC,56585A102,0001510295,2911,324110,10102030,101020,2006.0,2018.0


Marathon petrolium corp is a refinery spin-off of Marathon Oil (spun off in 2011). What we are interested in is the pipeline subsidiary of Maraton Oil (gvkey 007017).

In [41]:
db.raw_sql("SELECT * FROM compa.names WHERE lower(conm) LIKE '%%wolverine%%' AND year2 > 2000 LIMIT 10")

,gvkey,conm,tic,cusip,cik,sic,naics,gsubind,gind,year1,year2
0,011566,WOLVERINE WORLD WIDE,WWW,978097103,0000110471,3140,3162,25203020,252030,1960.0,2018.0
1,014450,WOLVERINE TUBE INC,WLVTQ,978093102,0000821407,3350,331421,20106020,201060,1987.0,2009.0
2,186290,WOLVERINE BANCORP INC,WBKC,977880103,0001500836,6035,522120,40102010,401020,2008.0,2016.0


The correct organization cannot be found here. We are probably looking at a wholly owned subsidiary, and can merge the observations with those of Marathon.

In [42]:
db.raw_sql("SELECT * FROM compa.names WHERE lower(conm) LIKE '%%tesoro%%' AND year2 > 2000 LIMIT 10")

,gvkey,conm,tic,cusip,cik,sic,naics,gsubind,gind,year1,year2
0,031655,TESORO CORP-PRO FORMA,TSO.P,88199G936,None,2911,324110,None,None,2015.0,2016.0
1,105634,TESORO MINERALS CORP,NNGVF,88160V102,None,1040,212221,15104030,151040,2002.0,2018.0


### Track asset values

#### Tesoro

In [43]:
db.raw_sql("""SELECT gvkey, datadate, fyear, revt AS revenue, act AS assets FROM compa.funda 
              WHERE gvkey IN ('031655', '105634') AND fyear BETWEEN '2014' AND '2017'
              AND (act is not null OR revt is not null)""")

,gvkey,datadate,fyear,revenue,assets
0,031655,2015-12-31,2015.0,38763.0,NaN
1,031655,2016-09-30,2016.0,23749.0,6783.000
2,105634,2014-10-31,2014.0,0.0,0.836
3,105634,2015-10-31,2015.0,0.0,0.556
4,105634,2016-10-31,2016.0,0.0,0.098
5,105634,2017-10-31,2017.0,0.0,0.345


#### Andeavor

Tesoro also used to be called andeavor.

In [44]:
db.raw_sql("SELECT * FROM compa.names WHERE lower(conm) LIKE '%%andeavor%%' AND year2 > 2000 LIMIT 10")

,gvkey,conm,tic,cusip,cik,sic,naics,gsubind,gind,year1,year2
0,010466,ANDEAVOR,ANDV,03349M105,0000050104,2911,324110,10102030,101020,1966.0,2017.0
1,186206,ANDEAVOR LOGISTICS LP,ANDX,03350F106,0001507615,4610,486,10102040,101020,2005.0,2018.0


In [45]:
db.raw_sql("""SELECT gvkey, datadate, fyear, revt AS revenue, act AS assets FROM compa.funda 
              WHERE gvkey IN ('010466', '186206') AND fyear>'2015'
              AND (act is not null OR revt is not null)""")

,gvkey,datadate,fyear,revenue,assets
0,010466,2016-12-31,2016.0,24005.0,7414.0
1,010466,2017-12-31,2017.0,34204.0,6883.0
2,186206,2016-12-31,2016.0,1220.0,938.0
3,186206,2017-12-31,2017.0,3213.0,505.0
4,186206,2018-12-31,2018.0,2380.0,563.0


#### Marathon

In [46]:
db.raw_sql("""SELECT gvkey, datadate, fyear, revt AS revenue, act AS assets FROM compa.funda WHERE gvkey='007017'
           AND fyear BETWEEN '2015' AND '2019' AND act is not null""")

,gvkey,datadate,fyear,revenue,assets
0,007017,2015-12-31,2015.0,5522.0,2590.0
1,007017,2016-12-31,2016.0,4031.0,3665.0
2,007017,2017-12-31,2017.0,4373.0,2566.0
3,007017,2018-12-31,2018.0,5888.0,2921.0


In [47]:
db.raw_sql("""SELECT gvkey, datadate, fyear, revt AS revenue, act AS assets FROM compa.funda WHERE gvkey='186989'
           AND fyear BETWEEN '2015' AND '2019' AND act is not null""")

,gvkey,datadate,fyear,revenue,assets
0,186989,2015-12-31,2015.0,64359.0,9471.0
1,186989,2016-12-31,2016.0,55833.0,10401.0
2,186989,2017-12-31,2017.0,66974.0,13401.0
3,186989,2018-12-31,2018.0,96504.0,18023.0


The asset gains 2016-2018 are roughly in line with the asset value of Tesoro.

### Decision

The merger between Marathon and Tesoro was finalized in 2018 (http://ir.andeavor.com/phoenix.zhtml?c=79122&p=irol-newsArticle&ID=2345407). We will group Wolverine with Marathon, and also combine the two Tesoros into one observation. For 2018, we will subsume Tesoro under Marathon.

We group Wolverine with Marathon, and the two Tesoros together. We cut of the observations for Tesoro group in 2016, and add their observations to the Marathon group, in order to track the impact of this acquisition as a source of change.

In [48]:
# Marathon and Wolverine
company_groups = add_company_group('Marathon (Group)', ['32147', '22830'], company_groups)

# Tesoro
company_groups = add_company_group('Tesoro (Group)', ['38933', '39029'], company_groups)
company_groups.tail()

,members,name
4,31371,Buckeye (Group)
5,32147,Marathon (Group)
6,22830,Marathon (Group)
7,38933,Tesoro (Group)
8,39029,Tesoro (Group)


In [49]:
m_as = add_m_a('Marathon (Group)', ['Marathon (Group)', 'Tesoro (Group)'], m_as, start_year = '2018')
m_as

,members,name,start_year
0,Marathon (Group),Marathon (Group),2018
1,Tesoro (Group),Marathon (Group),2018


### 8.2.6 Sunoco

In [50]:
print(next(issues))


SUNOCO PIPELINE L.P. (OPERATOR_ID 18718) has the same parent company as:

	MID - VALLEY PIPELINE CO (OPERATOR_ID 12470)

	ENERGY TRANSFER COMPANY (OPERATOR_ID 32099)

	DAPL-ETCO OPERATIONS MANAGEMENT, LLC (OPERATOR_ID 39205)

	PERMIAN EXPRESS PARTNERS LLC (OPERATOR_ID 39596)



### Names and miles

In [51]:
find_names('18718')

[{'name': 'SUNOCO PIPELINE L.P.', 'start_year': 2010, 'end_year': 2018}]

In [52]:
find_total_miles('18718', range(2010, 2019))

[[2010, 3466.74],
 [2011, 3379.19],
 [2012, 3098.0699999999997],
 [2013, 2529.64],
 [2014, 3185.63],
 [2015, 3364.17],
 [2016, 3308.1299999999997],
 [2017, 3320.93],
 [2018, 3050.2]]

In [53]:
find_names('12470')

[{'name': 'MID - VALLEY PIPELINE CO', 'start_year': 2010, 'end_year': 2018}]

In [54]:
find_total_miles('12470', range(2010, 2019))

[[2010, 542.0],
 [2011, 542.0],
 [2012, 542.0],
 [2013, 633.0],
 [2014, 656.0],
 [2015, 662.0],
 [2016, 632.0],
 [2017, 650.0],
 [2018, 650.47]]

A wholly owned subsidiary of Sunoco.

In [55]:
find_names('32099')

[{'name': 'ENERGY TRANSFER COMPANY', 'start_year': 2011, 'end_year': 2018}]

In [56]:
find_total_miles('32099', range(2010, 2019))

[[2010, 0.0],
 [2011, 331.8],
 [2012, 463.2],
 [2013, 463.43],
 [2014, 460.89],
 [2015, 468.79],
 [2016, 744.4200000000001],
 [2017, 1310.0],
 [2018, 1295.11]]

In [57]:
find_names('39205')

[{'name': 'DAPL-ETCO OPERATIONS MANAGEMENT, LLC',
  'start_year': 2017,
  'end_year': 2018}]

In [58]:
find_total_miles('39205', range(2010, 2019))

[[2010, 0.0],
 [2011, 0.0],
 [2012, 0.0],
 [2013, 0.0],
 [2014, 0.0],
 [2015, 0.0],
 [2016, 0.0],
 [2017, 795.9],
 [2018, 950.64]]

Dacota Access PipeLine - construction started in 2016.

In [59]:
find_names('39596')

[{'name': 'PERMIAN EXPRESS PARTNERS LLC',
  'start_year': 2017,
  'end_year': 2018}]

In [60]:
find_total_miles('39596', range(2010, 2019))

[[2010, 0.0],
 [2011, 0.0],
 [2012, 0.0],
 [2013, 0.0],
 [2014, 0.0],
 [2015, 0.0],
 [2016, 0.0],
 [2017, 946.0],
 [2018, 946.62]]

Permian express oil pipeline - wholly owned subsidiary of Sunoco, construction started in 2013 (https://www.sourcewatch.org/index.php/Permian_Express_Oil_Pipeline).

### Company structure - gvkeys on wrds

In [61]:
db.raw_sql("SELECT * FROM compa.names WHERE lower(conm) LIKE 'sunoco%%' LIMIT 10")

,gvkey,conm,tic,cusip,cik,sic,naics,gsubind,gind,year1,year2
0,010156,SUNOCO INC,SUN.1,86764P109,0000095304,2911,324110,10102030,101020,1950.0,2011.0
1,012892,SUNOCO LP,SUN,86765K109,0001552275,5172,424720,10102040,101020,2007.0,2018.0
2,147208,SUNOCO LOGISTICS PARTNERS LP,SXL,86764L108,None,4610,486110,10102040,101020,1997.0,2016.0


For sunoco, the third row is a good hit. The naics matches (crude oil pipelines), and the company history matches public information. 

In [62]:
db.raw_sql("SELECT * FROM compa.names WHERE lower(conm) LIKE 'energy transfer%%' LIMIT 10")

,gvkey,conm,tic,cusip,cik,sic,naics,gsubind,gind,year1,year2
0,141720,ENERGY TRANSFER PARTNERS -LP,ETP,29278N103,0001161154,4922,486210,10102040,101020,1999.0,2017.0
1,165649,ENERGY TRANSFER LP,ET,29273V100,0001276187,4922,486210,10102040,101020,2004.0,2018.0
2,170976,ENERGY TRANSFER EQY-PROFORMA,ETE.P,29299L938,None,4922,221210,10102040,101020,2010.0,2010.0


For energy transfer company, the second row matches. The first row is the parent company that currently holds both of them (https://www.wsj.com/articles/sunoco-logistics-to-buy-energy-transfer-owners-of-dakota-access-pipeline-1479742796). Sunoco was acquired by the parent in 2012, and acuired that parent's pipeline subsidiary in 2017.

In [63]:
db.raw_sql("SELECT * FROM compa.names WHERE lower(conm) LIKE '%%permian%%' LIMIT 10")

,gvkey,conm,tic,cusip,cik,sic,naics,gsubind,gind,year1,year2
0,008493,PERMIAN BASIN ROYALTY TRUST,PBT,714236106,0000319654,6792,533110,10102020,101020,1981.0,2018.0
1,013575,PERMIAN PARTNERS -LP,5706B,714240207,0000812023,5172,422720,10102030,101020,1983.0,1989.0
2,019171,RSP PERMIAN INC,RSPP,74978Q105,0001588216,1311,211111,10102020,101020,2011.0,2017.0


Based on the industry classification and years of existence, the first and third are all potential candidates.

In [64]:
db.raw_sql("SELECT * FROM compa.names WHERE (lower(conm) LIKE '%%dapl%%' OR lower(conm) LIKE '%%dakota%%access%%') LIMIT 10")

,gvkey,conm,tic,cusip,cik,sic,naics,gsubind,gind,year1,year2


In [65]:
db.raw_sql("SELECT * FROM compa.names WHERE lower(conm) LIKE '%%mid%%-%%-valley%%' LIMIT 10")

,gvkey,conm,tic,cusip,cik,sic,naics,gsubind,gind,year1,year2


Information on these should be found as subsidiaries.

### Company financials on wrds

#### Sunoco

In [66]:
db.raw_sql("""SELECT gvkey, datadate, fyear, revt AS revenue, act AS assets  
              FROM compa.funda WHERE gvkey IN ('012892', '147208')
              AND fyear between '2011' AND '2018'
              AND (act is not null OR revt is not null)""")

,gvkey,datadate,fyear,revenue,assets
0,012892,2011-12-31,2011.0,3820.378,147.412
1,012892,2012-12-31,2012.0,4277.341,103.105
2,012892,2013-12-31,2013.0,4492.579,138.222
3,012892,2014-12-31,2014.0,5382.016,225.141
4,012892,2015-12-31,2015.0,16935.347,1014.151
5,012892,2016-12-31,2016.0,15698.000,1389.000
6,012892,2017-12-31,2017.0,11723.000,4544.000
7,012892,2018-12-31,2018.0,16994.000,905.000
8,147208,2011-12-31,2011.0,10905.000,2506.000
9,147208,2012-12-31,2012.0,13110.000,2390.000


#### Energy Transfer

In [67]:
db.raw_sql("""SELECT gvkey, datadate, fyear, revt AS revenue, act AS assets  
              FROM compa.funda WHERE gvkey='165649'
              AND fyear between '2011' AND '2018'
              AND (act is not null OR revt is not null)""")

,gvkey,datadate,fyear,revenue,assets
0,165649,2011-12-31,2011.0,8240.703,1455.444
1,165649,2012-12-31,2012.0,16391.000,5597.000
2,165649,2013-12-31,2013.0,46115.000,6536.000
3,165649,2014-12-31,2014.0,53231.000,6153.000
4,165649,2015-12-31,2015.0,39076.000,5410.000
5,165649,2016-12-31,2016.0,34024.000,6985.000
6,165649,2017-12-31,2017.0,40289.000,10683.000
7,165649,2018-12-31,2018.0,53717.000,6750.000


In [68]:
db.raw_sql("""SELECT gvkey, datadate, fyear, revt AS revenue, act AS assets  
              FROM compa.funda WHERE gvkey in ('141720', '165649', '170976')
              AND fyear between '2011' AND '2018'
              AND (act is not null OR revt is not null)""")

,gvkey,datadate,fyear,revenue,assets
0,141720,2011-12-31,2011.0,6850.440,1275.494
1,141720,2012-12-31,2012.0,15129.000,5404.000
2,141720,2013-12-31,2013.0,44119.000,6239.000
3,141720,2014-12-31,2014.0,48698.000,5439.000
4,141720,2015-12-31,2015.0,32442.000,4698.000
5,141720,2016-12-31,2016.0,21827.000,5729.000
6,141720,2017-12-31,2017.0,29054.000,6528.000
7,165649,2011-12-31,2011.0,8240.703,1455.444
8,165649,2012-12-31,2012.0,16391.000,5597.000
9,165649,2013-12-31,2013.0,46115.000,6536.000


The numbers match up. The act column describes total asset value, we see that the assets added to ENERGY TRANSFER LP (gvkey 165649) in 2017 roughly equal those of the asset value from 2016 combined with the assets from SUNOCO.

We can retain both organizations until 2016 in our dataset and be fairly certain that we will be able to find financial data from capital iq on the organization also.

### Decision

We list Sunoco (OPERATOR_ID 18718), Mid-Valley Pipeline (OPERATOR_ID 12470), and Permian (OPERATOR_ID 39596) together as one company until 2017. Similarly, we list Energy Transfer (OPERATOR_ID 32099) and DAPL (OPERATOR_ID 39205) together until 2017. From 2017, we merge the two under the name of Sunoco.

In [69]:
# Sunoco
company_groups = add_company_group('Sunoco (Group)', ['18718', '12470', '39596'], company_groups)

# Tesoro
company_groups = add_company_group('Energy Transfer (Group)', ['32099', '39205'], company_groups)
company_groups.tail()

,members,name
9,18718,Sunoco (Group)
10,12470,Sunoco (Group)
11,39596,Sunoco (Group)
12,32099,Energy Transfer (Group)
13,39205,Energy Transfer (Group)


In [70]:
m_as = add_m_a('Sunoco (Group)', ['Sunoco (Group)', 'Energy Transfer (Group)'], m_as, start_year = '2017')
m_as

,members,name,start_year
0,Marathon (Group),Marathon (Group),2018
1,Tesoro (Group),Marathon (Group),2018
2,Sunoco (Group),Sunoco (Group),2017
3,Energy Transfer (Group),Sunoco (Group),2017


### 8.2.7 ExxonMobil

In [71]:
print(next(issues))


EXXONMOBIL PIPELINE CO (OPERATOR_ID 4906) has the same parent company as:

	EXXONMOBIL PIPELINE CO (OPERATOR_ID 12628)



In [72]:
find_names('4906')

[{'name': 'EXXONMOBIL PIPELINE CO', 'start_year': 2010, 'end_year': 2018}]

In [73]:
find_names('12628')

[{'name': 'EXXONMOBIL PIPELINE CO', 'start_year': 2010, 'end_year': 2018},
 {'name': 'MOBIL  PIPE  LINE COMPANY', 'start_year': 2012, 'end_year': 2018},
 {'name': 'MOBIL PIPELINE CO', 'start_year': 2010, 'end_year': 2010}]

Company name inconsistent between different documents.

In [74]:
db.raw_sql("SELECT * FROM compa.names WHERE lower(conm) LIKE '%%mobil pipe%%' LIMIT 10")

,gvkey,conm,tic,cusip,cik,sic,naics,gsubind,gind,year1,year2


All are wholly owned subsidiaries of exxonmobil. We combine them into one observation.

In [75]:
company_groups = add_company_group('ExxonMobil (Group)', ['4906', '12628'], company_groups)
company_groups.tail()

,members,name
11,39596,Sunoco (Group)
12,32099,Energy Transfer (Group)
13,39205,Energy Transfer (Group)
14,4906,ExxonMobil (Group)
15,12628,ExxonMobil (Group)


### 8.2.8 Flint Hills

In [76]:
print(next(issues))


FLINT HILLS RESOURCES, LC (OPERATOR_ID 22855) has changed its name:

	Was named FLINT HILLS RESOURCES, LC from 2017 to 2018.

	Was named KOCH PIPELINE COMPANY, L.P. from 2010 to 2016.



In [77]:
find_total_miles('22855', range(2015, 2019))

[[2015, 1885.19], [2016, 1674.88], [2017, 1711.48], [2018, 1750.7199999999998]]

The restructuring or sell-off does not seem to affect the organizations pipeline assets. We will ignore the rebranding efforts.

### 8.2.9 Nustar

In [78]:
print(next(issues))


NUSTAR PIPELINE OPERATING PARTNERSHIP L.P. (OPERATOR_ID 10012) has the same parent company as:

	NUSTAR LOGISTICS, L.P. (OPERATOR_ID 31454)



In [79]:
find_names('10012')

[{'name': 'NUSTAR PIPELINE OPERATING PARTNERSHIP L.P.',
  'start_year': 2010,
  'end_year': 2018}]

In [80]:
find_names('31454')

[{'name': 'NUSTAR LOGISTICS, L.P.', 'start_year': 2010, 'end_year': 2018}]

In [81]:
db.raw_sql("SELECT * FROM compa.names WHERE lower(conm) LIKE '%%nustar%%' LIMIT 10")

,gvkey,conm,tic,cusip,cik,sic,naics,gsubind,gind,year1,year2
0,107044,INTREPID NUSTAR EXCHANGE CP,IXN.,461193104,None,1000,212,15104040,151040,2001.0,2005.0
1,141363,NUSTAR ENERGY LP,NS,67058H102,0001110805,2911,324110,10102030,101020,1997.0,2018.0
2,174135,NUSTAR GP HOLDINGS LLC,NSH,67059L102,0001223786,5171,424710,10102040,101020,2001.0,2017.0


Nustar GP was a holding in Europe (https://www.newswire.ca/news-releases/inter-pipeline-announces-acquisition-of-multinational-european-bulk-liquid-storage-business-698997581.html), so it is not the organization that we see in the data. Since the two organizations are not listed separately we will consolidate them into one organization.

In [82]:
company_groups = add_company_group('NuStar', ['10012', '31454'], company_groups)
company_groups.tail()

,members,name
13,39205,Energy Transfer (Group)
14,4906,ExxonMobil (Group)
15,12628,ExxonMobil (Group)
16,10012,NuStar
17,31454,NuStar


### 8.2.10 Kinder Morgan

In [83]:
print(next(issues))


PLANTATION PIPE LINE CO (OPERATOR_ID 15674) has the same parent company as:

	SFPP, LP (OPERATOR_ID 18092)

	CALNEV PIPELINE CO (OPERATOR_ID 26125)

	KINDER MORGAN COCHIN LLC (OPERATOR_ID 32258)



In [84]:
find_names('15674')

[{'name': 'PLANTATION PIPE LINE CO', 'start_year': 2010, 'end_year': 2018}]

Owned by Kinder Morgan since 2000 (https://www.kindermorgan.com/pages/business/products_pipelines/plantation_history.aspx).

In [85]:
find_names('18092')

[{'name': 'SFPP, LP', 'start_year': 2010, 'end_year': 2018}]

SFPP is a wholly owned subsidiary for the observation period also.

In [86]:
find_names('26125')

[{'name': 'CALNEV PIPELINE CO', 'start_year': 2010, 'end_year': 2018}]

Also wholly owned for observation period.

In [87]:
find_names('32258')

[{'name': 'KINDER MORGAN COCHIN LLC', 'start_year': 2010, 'end_year': 2018}]

Also wholly owned for observation period.

### Decision

In [88]:
company_groups = add_company_group('Kinder Morgan (Group)', ['15674', '18092', '26125', '32258'], company_groups)
company_groups.tail()

,members,name
17,31454,NuStar
18,15674,Kinder Morgan (Group)
19,18092,Kinder Morgan (Group)
20,26125,Kinder Morgan (Group)
21,32258,Kinder Morgan (Group)


### 8.2.11 Enbridge

In [89]:
print(next(issues))


ENBRIDGE ENERGY, LIMITED PARTNERSHIP (OPERATOR_ID 11169) has the same parent company as:

	EXPRESS HOLDINGS (USA), LLC (OPERATOR_ID 31720)

	CCPS TRANSPORTATION, LLC (OPERATOR_ID 32080)

	ENBRIDGE PIPELINES (SOUTHERN LIGHTS) L.L.C. (OPERATOR_ID 32502)



In [90]:
find_names('11169')

[{'name': 'ENBRIDGE ENERGY, LIMITED PARTNERSHIP',
  'start_year': 2010,
  'end_year': 2018}]

In [91]:
find_names('31720')

[{'name': 'EXPRESS HOLDINGS (USA), LLC', 'start_year': 2013, 'end_year': 2018},
 {'name': 'KINDER MORGAN PIPELINES (USA) INC',
  'start_year': 2010,
  'end_year': 2012}]

In [92]:
find_total_miles('31720', range(2010, 2015))

[[2010, 196.0], [2011, 196.0], [2012, 196.0], [2013, 219.3], [2014, 212.4]]

In [93]:
find_names('32080')

[{'name': 'CCPS TRANSPORTATION, LLC', 'start_year': 2010, 'end_year': 2018}]

CCPS has been a wholly owned subsidiary of Enbridge since 2005

In [94]:
find_names('32502')

[{'name': 'ENBRIDGE PIPELINES (SOUTHERN LIGHTS) L.L.C.',
  'start_year': 2010,
  'end_year': 2018}]

### Express Holdings

In [95]:
find_total_miles('31720', range(2010, 2015))

[[2010, 196.0], [2011, 196.0], [2012, 196.0], [2013, 219.3], [2014, 212.4]]

The Express pipeline system was sold by Kinder Morgan to Spectra Energy ~2012-2013 (https://ir.kindermorgan.com/sites/kindermorgan.investorhq.businesswire.com/files/report/additional/KMI_2012_annual_report_financials.pdf). Enbridge merged with Spectra Energy in 2018 (https://www.newswire.ca/news-releases/enbridge-inc-and-spectra-energy-partners-lp-complete-merger-702921861.html).

### CCPS Transportation

Has been acquired (90%) by Enbridge in 2003 (https://apps.neb-one.gc.ca/REGDOCS/File/Download/346893).

### Decision

We group the CCPS and Enbridge Pipelines (Southern Lights) with Enbridge. Express Holdings is modeled as part of Kinder Morgan until 2012, then as part of Spectra Energy (which has no other pipeline business in the datset) until 2017, And finally as part of Enbridge in 2018.

In [96]:
company_groups = add_company_group('Enbridge (Group)', ['11169', '32080', '32502'], company_groups)
company_groups.tail()

,members,name
20,26125,Kinder Morgan (Group)
21,32258,Kinder Morgan (Group)
22,11169,Enbridge (Group)
23,32080,Enbridge (Group)
24,32502,Enbridge (Group)


In [97]:
# new_9 - Kinder Morgan
m_as = add_m_a('Kinder Morgan (Group)', ['Kinder Morgan (Group)', '31720'], m_as, end_year = '2013')

m_as = add_m_a('Enbridge (Group)', ['Enbridge (Group)', '31720'], m_as, start_year = '2018')
m_as

,members,name,start_year,end_year
0,Marathon (Group),Marathon (Group),2018,NaN
1,Tesoro (Group),Marathon (Group),2018,NaN
2,Sunoco (Group),Sunoco (Group),2017,NaN
3,Energy Transfer (Group),Sunoco (Group),2017,NaN
4,Kinder Morgan (Group),Kinder Morgan (Group),NaN,2013
5,31720,Kinder Morgan (Group),NaN,2013
6,Enbridge (Group),Enbridge (Group),2018,NaN
7,31720,Enbridge (Group),2018,NaN


### 8.2.11 Plains Pipeline

In [98]:
print(next(issues))


PLAINS PIPELINE, L.P. (OPERATOR_ID 300) has the same parent company as:

	ROCKY MOUNTAIN PIPELINE SYSTEM, LLC (OPERATOR_ID 31666)



Wholly owned subsidiary for the duration of our observation period.

In [99]:
company_groups = add_company_group('Plains Pipeline (Group)', ['300', '31666'], company_groups)
company_groups.tail()

,members,name
22,11169,Enbridge (Group)
23,32080,Enbridge (Group)
24,32502,Enbridge (Group)
25,300,Plains Pipeline (Group)
26,31666,Plains Pipeline (Group)


### 8.2.12 BP

In [100]:
print(next(issues))


BP PIPELINE (NORTH AMERICA) INC. (OPERATOR_ID 31189) has the same parent company as:

	OLYMPIC PIPE LINE COMPANY (OPERATOR_ID 30781)



BP owned olympic for the duration of our observation period.

In [101]:
find_names('30781')

[{'name': 'OLYMPIC PIPE LINE COMPANY', 'start_year': 2010, 'end_year': 2018}]

In [102]:
find_total_miles('30781', range(2010, 2019))

[[2010, 316.0],
 [2011, 316.0],
 [2012, 316.0],
 [2013, 366.0],
 [2014, 380.3],
 [2015, 380.4],
 [2016, 380.4],
 [2017, 380.7],
 [2018, 380.6]]

In [103]:
company_groups = add_company_group('BP (Group)', ['31189', '30781'], company_groups)
company_groups.tail()

,members,name
24,32502,Enbridge (Group)
25,300,Plains Pipeline (Group)
26,31666,Plains Pipeline (Group)
27,31189,BP (Group)
28,30781,BP (Group)


### 8.2.13

In [104]:
print(next(issues))


EXXONMOBIL PIPELINE CO (OPERATOR_ID 12628) has changed its name:

	Was named EXXONMOBIL PIPELINE CO from 2010 to 2018.

	Was named MOBIL  PIPE  LINE COMPANY from 2012 to 2018.

	Was named MOBIL PIPELINE CO from 2010 to 2010.



The different names used do not imply an ownership change.

### 8.2.14

In [105]:
print(next(issues))


TESORO LOGISTICS OPERATIONS, LLC (OPERATOR_ID 38933) has changed its name:

	Was named TESORO LOGISTICS OPERATIONS LLC - SOUTHERN CALIFORNIA from 2012 to 2017.

	Was named TESORO LOGISTICS OPERATIONS, LLC from 2018 to 2018.



In [106]:
find_total_miles('38933', range (2010, 2019))

[[2010, 0.0],
 [2011, 0.0],
 [2012, 15.5],
 [2013, 15.5],
 [2014, 15.5],
 [2015, 15.3],
 [2016, 15.5],
 [2017, 14.969999999999999],
 [2018, 930.6999999999999]]

We have already covered this organization above.

### 8.2.15

In [107]:
print(next(issues))


TC OIL PIPELINE OPERATIONS INC (OPERATOR_ID 32334) may have a namesake or namesakes:

	DAPL-ETCO OPERATIONS MANAGEMENT, LLC (OPERATOR_ID 39205)



No action necessary.

### 8.2.16

In [108]:
print(next(issues))


DIXIE PIPELINE COMPANY LLC (OPERATOR_ID 3445) has changed its name:

	Was named DIXIE PIPELINE from 2010 to 2010.

	Was named DIXIE PIPELINE COMPANY LLC from 2011 to 2018.



No action necessary.

### 8.2.17

In [109]:
print(next(issues))


KINDER MORGAN COCHIN LLC (OPERATOR_ID 32258) may have a namesake or namesakes:

	KINDER MORGAN PIPELINES (USA) INC (OPERATOR_ID 31720)



In [110]:
find_names('32258')

[{'name': 'KINDER MORGAN COCHIN LLC', 'start_year': 2010, 'end_year': 2018}]

In [111]:
find_names('31720')

[{'name': 'EXPRESS HOLDINGS (USA), LLC', 'start_year': 2013, 'end_year': 2018},
 {'name': 'KINDER MORGAN PIPELINES (USA) INC',
  'start_year': 2010,
  'end_year': 2012}]

We have already handled this one above.

### 8.2.18

In [112]:
print(next(issues))


TESORO LOGISTICS NORTHWEST PIPELINE LLC (OPERATOR_ID 39029) may have a namesake or namesakes:

	TESORO LOGISTICS OPERATIONS LLC - SOUTHERN CALIFORNIA (OPERATOR_ID 38933)



Already addressed.

### 8.2.19

In [113]:
print(next(issues))


EXPRESS HOLDINGS (USA), LLC (OPERATOR_ID 31720) has changed its name:

	Was named EXPRESS HOLDINGS (USA), LLC from 2013 to 2018.

	Was named KINDER MORGAN PIPELINES (USA) INC from 2010 to 2012.

EXPRESS HOLDINGS (USA), LLC (OPERATOR_ID 31720) may have a namesake or namesakes:

	TALLGRASS PONY EXPRESS PIPELINE, LLC (OPERATOR_ID 39043)

	PERMIAN EXPRESS PARTNERS LLC (OPERATOR_ID 39596)

	KINDER MORGAN COCHIN LLC (OPERATOR_ID 32258)



We have already addressed this above.

## 8.3 Overview

In [114]:
company_groups

,members,name
0,31618,Enterprise Products (Group)
1,30829,Enterprise Products (Group)
2,3445,Enterprise Products (Group)
3,1845,Buckeye (Group)
4,31371,Buckeye (Group)
5,32147,Marathon (Group)
6,22830,Marathon (Group)
7,38933,Tesoro (Group)
8,39029,Tesoro (Group)
9,18718,Sunoco (Group)


In [115]:
m_as

,members,name,start_year,end_year
0,Marathon (Group),Marathon (Group),2018,NaN
1,Tesoro (Group),Marathon (Group),2018,NaN
2,Sunoco (Group),Sunoco (Group),2017,NaN
3,Energy Transfer (Group),Sunoco (Group),2017,NaN
4,Kinder Morgan (Group),Kinder Morgan (Group),NaN,2013
5,31720,Kinder Morgan (Group),NaN,2013
6,Enbridge (Group),Enbridge (Group),2018,NaN
7,31720,Enbridge (Group),2018,NaN


In [116]:
spin_offs

[{'name': '31684', 'year': '2011'}]

In [117]:
company_groups.to_feather(f'../preprocessed_data/company_groups_{today}.feather')
m_as.to_feather(f'../preprocessed_data/m_as_{today}.feather')